In [1]:
import sqlite3
import pandas
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn import ensemble
import numpy
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import  GridSearchCV
from sklearn import tree
import matplotlib.pyplot as plt


# including our functions from last week up here for use. 
def cleanup_data(df, cutoffPercent = .01):
    for col in df:
        sizes = df[col].value_counts(normalize = True)
        # get the names of the levels that make up less than 1% of the dataset
        values_to_delete = sizes[sizes<cutoffPercent].index
        df[col].ix[df[col].isin(values_to_delete)] = "Other"
    return df
#

def get_binary_values(data_frame):
    """encodes cateogrical features in Pandas.
    """
    all_columns = pandas.DataFrame( index = data_frame.index)
    for col in data_frame.columns:
        data = pandas.get_dummies(data_frame[col], prefix=col.encode('ascii', 'replace'))
        all_columns = pandas.concat([all_columns, data], axis=1)
    return all_columns
#
def find_zero_var(df):
    """finds columns in the dataframe with zero variance -- ie those
        with the same value in every observation.
    """   
    toKeep = []
    toDelete = []
    for col in df:
        if len(df[col].value_counts()) > 1:
            toKeep.append(col)
        else:
            toDelete.append(col)
        ##
    return {'toKeep':toKeep, 'toDelete':toDelete} 
##
def find_perfect_corr(df):
    """finds columns that are eother positively or negatively perfectly correlated (with correlations of +1 or -1), and creates a dict 
        that includes which columns to drop so that each remaining column
        is independent
    """  
    corrMatrix = df.corr()
    corrMatrix.loc[:,:] =  numpy.tril(corrMatrix.values, k = -1)
    already_in = set()
    result = []
    for col in corrMatrix:
        perfect_corr = corrMatrix[col][abs(numpy.round(corrMatrix[col],10)) == 1.00].index.tolist()
        if perfect_corr and col not in already_in:
            already_in.update(set(perfect_corr))
            perfect_corr.append(col)
            result.append(perfect_corr)
    toRemove = []
    for item in result:
        toRemove.append(item[1:(len(item)+1)])
    toRemove = sum(toRemove, [])
    return {'corrGroupings':result, 'toRemove':toRemove}
###


# putting a setting into pandas that lets you print out the entire
# DataFrame when you use the .head() method
pandas.set_option('display.max_columns', None)

#Use SQL to extract relevant data from SQLite Database

In [23]:
## Query that joins the batting, pitching and fielding tables
monster_query = """
select m.nameGiven, d.teamID, m.weight, m.height, m.bats, m.throws, hfi.inducted, batting.*, pitching.*, fielding.* from hall_of_fame_inductees hfi 
left outer join master m on hfi.playerID = m.playerID
left outer join 
(
select playerID,  sum(AB) as total_at_bats, sum(H) as total_hits, sum(R) as total_runs, sum(HR) as total_home_runs, sum(SB) as total_stolen_bases,
sum(RBI) as total_RBI, sum(CS) as total_caught_stealing, sum(SO) as total_hitter_strikeouts, sum(IBB) as total_intentional_walks
from Batting
group by playerID
HAVING max(yearID) > 1950 and min(yearID) >1950 
)
batting on batting.playerID = hfi.playerID
left outer join
(
 select playerID, sum(G) as total_games_pitched, sum(SO) as total_shutouts, sum(sv) as total_saves, sum(IPouts) as total_outs_pitched, 
sum(H) as total_pitching_hits, sum(er) as total_pitching_earned_runs, sum(so) as total_pitcher_strikeouts, 
avg(ERA) as average_ERA, sum(WP) as total_wild_pitches, sum(HBP) as total_hit_by_pitch, sum(GF) as total_games_finished,
sum(R) as total_runs_allowed
from Pitching
group by playerID
) 
pitching on pitching.playerID = hfi.playerID 
LEFT OUTER JOIN
(
select playerID, sum(G) as total_games_fielded, sum(InnOuts) as total_time_in_field_with_outs, 
sum(PO) as total_putouts, sum(E) as total_errors, sum(DP) as total_double_plays
from Fielding
group by playerID
) 
fielding on fielding.playerID = hfi.playerID

LEFT OUTER JOIN dominant_team_per_player d on d.playerID = hfi.playerID
where batting.playerID is not null
"""

DATABASE = r'lahman2013.sqlite'
con = sqlite3.connect(DATABASE)
df = pandas.read_sql(monster_query, con)
con.close()

## getting an intial view of the data for validation
df.head(10)
df.columns

Index([u'nameGiven', u'teamID', u'weight', u'height', u'bats', u'throws', u'inducted', u'playerID', u'total_at_bats', u'total_hits', u'total_runs', u'total_home_runs', u'total_stolen_bases', u'total_RBI', u'total_caught_stealing', u'total_hitter_strikeouts', u'total_intentional_walks', u'playerID', u'total_games_pitched', u'total_shutouts', u'total_saves', u'total_outs_pitched', u'total_pitching_hits', u'total_pitching_earned_runs', u'total_pitcher_strikeouts', u'average_ERA', u'total_wild_pitches', u'total_hit_by_pitch', u'total_games_finished', u'total_runs_allowed', u'playerID', u'total_games_fielded', u'total_time_in_field_with_outs', u'total_putouts', u'total_errors', u'total_double_plays'], dtype='object')

#Process Data for Regression

In [29]:
#dropping duplicate playerID columns
df = df.drop(['playerID'], axis=1).join(df['playerID'].ix[:, 0:1])
df.columns

Index([u'nameGiven', u'teamID', u'weight', u'height', u'bats', u'throws', u'inducted', u'total_at_bats', u'total_hits', u'total_runs', u'total_home_runs', u'total_stolen_bases', u'total_RBI', u'total_caught_stealing', u'total_hitter_strikeouts', u'total_intentional_walks', u'total_games_pitched', u'total_shutouts', u'total_saves', u'total_outs_pitched', u'total_pitching_hits', u'total_pitching_earned_runs', u'total_pitcher_strikeouts', u'average_ERA', u'total_wild_pitches', u'total_hit_by_pitch', u'total_games_finished', u'total_runs_allowed', u'total_games_fielded', u'total_time_in_field_with_outs', u'total_putouts', u'total_errors', u'total_double_plays', u'playerID'], dtype='object')

In [31]:
df.set_index('playerID',inplace=True)
df.head()

,nameGiven,teamID,weight,height,bats,throws,inducted,total_at_bats,total_hits,total_runs,total_home_runs,total_stolen_bases,total_RBI,total_caught_stealing,total_hitter_strikeouts,total_intentional_walks,total_games_pitched,total_shutouts,total_saves,total_outs_pitched,total_pitching_hits,total_pitching_earned_runs,total_pitcher_strikeouts,average_ERA,total_wild_pitches,total_hit_by_pitch,total_games_finished,total_runs_allowed,total_games_fielded,total_time_in_field_with_outs,total_putouts,total_errors,total_double_plays
playerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
aaronha01,Henry Louis,ML1,180,72,R,R,1,12364,3771,2174,755,240,2297,73,1383,293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6019,150488,12926,259,258
ageeto01,Tommie Lee,NYN,195,71,R,R,0,3912,999,558,130,167,433,81,918,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2201,52930,4759,123,36
akerja01,Jackie Delane,NYA,190,74,R,R,0,92,7,3,0,0,4,0,51,0,495,404,123,2238,679,272,404,3.854286,13,40,321,312,495,2239,65,8,20
alexado01,Doyle Lafayette,BAL,190,75,R,R,0,265,44,19,0,0,17,0,77,0,561,1528,3,10103,3376,1406,1528,3.854783,74,53,56,1541,561,10103,264,32,36
allenbe01,Bernard Keith,MIN,175,72,L,R,0,3404,815,357,73,13,351,16,424,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1025,24076,2031,100,558


In [40]:
#############
## repeating the same preprocessing from the previous lesson
############

## splitting out the explanatory features 
explanatory_features = [col for col in df.columns if col not in ['nameGiven', 'inducted']]
explanatory_df = df[explanatory_features]

# dropping rows with no data.
explanatory_df.dropna(how='all', inplace = True) 

# extracting column names 
explanatory_colnames = explanatory_df.columns

## doing the same for response
response_series = df.inducted
response_series.dropna(how='all', inplace = True) 

## seeing which explanatory feature rows got removed.  Looks like none.
response_series.index[~response_series.index.isin(explanatory_df.index)]



/Users/tdong/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Index([], dtype='object')

In [41]:
### now, let's seperate the numeric explanatory data from the string data
string_features = explanatory_df.ix[:, explanatory_df.dtypes == 'object']
numeric_features = explanatory_df.ix[:, explanatory_df.dtypes != 'object']

print numeric_features.head()
print string_features.head()


           weight  height  total_at_bats  total_hits  total_runs  \
playerID                                                           
aaronha01     180      72          12364        3771        2174   
ageeto01      195      71           3912         999         558   
akerja01      190      74             92           7           3   
alexado01     190      75            265          44          19   
allenbe01     175      72           3404         815         357   

           total_home_runs  total_stolen_bases  total_RBI  \
playerID                                                    
aaronha01              755                 240       2297   
ageeto01               130                 167        433   
akerja01                 0                   0          4   
alexado01                0                   0         17   
allenbe01               73                  13        351   

           total_caught_stealing  total_hitter_strikeouts  \
playerID                          

In [42]:

# that are all NANs, as they will show up as all 'Nothing' when we start binning or look for features with no variation)
string_features = string_features.fillna('Nothing')
# cleaning up string features
string_features = cleanup_data(string_features)
# binarizing string features 
encoded_data = get_binary_values(string_features)
## imputing features
imputer_object = Imputer(missing_values='NaN', strategy='median', axis=0)
imputer_object.fit(numeric_features)
numeric_features = pandas.DataFrame(imputer_object.transform(numeric_features), columns = numeric_features.columns,index=numeric_features.index)

## pulling together numeric and encoded data.
explanatory_df = pandas.concat([numeric_features, encoded_data],axis = 1)
explanatory_df.head()

,weight,height,total_at_bats,total_hits,total_runs,total_home_runs,total_stolen_bases,total_RBI,total_caught_stealing,total_hitter_strikeouts,total_intentional_walks,total_games_pitched,total_shutouts,total_saves,total_outs_pitched,total_pitching_hits,total_pitching_earned_runs,total_pitcher_strikeouts,average_ERA,total_wild_pitches,total_hit_by_pitch,total_games_finished,total_runs_allowed,total_games_fielded,total_time_in_field_with_outs,total_putouts,total_errors,total_double_plays,teamID_ATL,teamID_BAL,teamID_BOS,teamID_CAL,teamID_CHA,teamID_CHN,teamID_CIN,teamID_CLE,teamID_DET,teamID_HOU,teamID_KCA,teamID_LAN,teamID_MIN,teamID_ML1,teamID_ML4,teamID_MON,teamID_NYA,teamID_NYN,teamID_OAK,teamID_Other,teamID_PHI,teamID_PIT,teamID_SDN,teamID_SFN,teamID_SLN,teamID_TEX,teamID_WS2,bats_B,bats_L,bats_R,throws_L,throws_R
playerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
aaronha01,180,72,12364,3771,2174,755,240,2297,73,1383,293,453.5,1262,7.5,6265.5,1965,809,1262,3.755000,56.5,36.5,39,908,6019,150488,12926,259,258,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
ageeto01,195,71,3912,999,558,130,167,433,81,918,26,453.5,1262,7.5,6265.5,1965,809,1262,3.755000,56.5,36.5,39,908,2201,52930,4759,123,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
akerja01,190,74,92,7,3,0,0,4,0,51,0,495.0,404,123.0,2238.0,679,272,404,3.854286,13.0,40.0,321,312,495,2239,65,8,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
alexado01,190,75,265,44,19,0,0,17,0,77,0,561.0,1528,3.0,10103.0,3376,1406,1528,3.854783,74.0,53.0,56,1541,561,10103,264,32,36,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
allenbe01,175,72,3404,815,357,73,13,351,16,424,52,453.5,1262,7.5,6265.5,1965,809,1262,3.755000,56.5,36.5,39,908,1025,24076,2031,100,558,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [43]:
#now, let's find features with no variance 
no_variation = find_zero_var(explanatory_df)
print no_variation
explanatory_df.drop(no_variation['toDelete'], inplace = True)
#nothing to delete

{'toKeep': ['weight', 'height', 'total_at_bats', 'total_hits', 'total_runs', 'total_home_runs', 'total_stolen_bases', 'total_RBI', 'total_caught_stealing', 'total_hitter_strikeouts', 'total_intentional_walks', 'total_games_pitched', 'total_shutouts', 'total_saves', 'total_outs_pitched', 'total_pitching_hits', 'total_pitching_earned_runs', 'total_pitcher_strikeouts', 'average_ERA', 'total_wild_pitches', 'total_hit_by_pitch', 'total_games_finished', 'total_runs_allowed', 'total_games_fielded', 'total_time_in_field_with_outs', 'total_putouts', 'total_errors', 'total_double_plays', u'teamID_ATL', u'teamID_BAL', u'teamID_BOS', u'teamID_CAL', u'teamID_CHA', u'teamID_CHN', u'teamID_CIN', u'teamID_CLE', u'teamID_DET', u'teamID_HOU', u'teamID_KCA', u'teamID_LAN', u'teamID_MIN', u'teamID_ML1', u'teamID_ML4', u'teamID_MON', u'teamID_NYA', u'teamID_NYN', u'teamID_OAK', 'teamID_Other', u'teamID_PHI', u'teamID_PIT', u'teamID_SDN', u'teamID_SFN', u'teamID_SLN', u'teamID_TEX', u'teamID_WS2', u'bats_B'

In [44]:
# deleting perfect correlation
no_correlation = find_perfect_corr(explanatory_df)
print no_correlation
explanatory_df.drop(no_correlation['toRemove'], 1, inplace = True)

{'corrGroupings': [['total_pitcher_strikeouts', 'total_shutouts'], [u'throws_R', u'throws_L']], 'toRemove': ['total_shutouts', u'throws_L']}


In [45]:
# scaling data
scaler = preprocessing.StandardScaler()
scaler.fit(explanatory_df)
explanatory_df_scaled = pandas.DataFrame(scaler.transform(explanatory_df), columns = explanatory_df.columns,index=explanatory_df.index)

In [47]:
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [49]:
response_series.value_counts()

0    355
1     41
dtype: int64

In [51]:
response_series.head()

playerID
aaronha01    1
ageeto01     0
akerja01     0
alexado01    0
allenbe01    0
Name: inducted, dtype: int64

In [50]:
#try to run logistic regression on unscaled model first
logit = sm.Logit(response_series, explanatory_df_scaled)
 
# fit the model
result = logit.fit()

LinAlgError: Singular matrix

In [59]:
def independent_columns(A, tol = 0):#1e-05):
    """
    Return an array composed of independent columns of A.

    Note the answer may not be unique; this function returns one of many
    possible answers.

    http://stackoverflow.com/q/13312498/190597 (user1812712)
    http://math.stackexchange.com/a/199132/1140 (Gerry Myerson)
    http://mail.scipy.org/pipermail/numpy-discussion/2008-November/038705.html
        (Anne Archibald)

    >>> A = np.array([(2,4,1,3),(-1,-2,1,0),(0,0,2,2),(3,6,2,5)])
    2 4 1 3
    -1 -2 1 0
    0 0 2 2
    3 6 2 5
    # try with checking the rank of matrixs 
    >>> independent_columns(A)
    np.array([[1, 4],
              [2, 5],
              [3, 6]])
    """
    Q, R = np.linalg.qr(A)
    independent = np.where(np.abs(R.diagonal()) > tol)[0]
    #print independent
    return A[:, independent], independent


A,independent_col_indexes=independent_columns(explanatory_df.as_matrix(columns=None)) 
#train_cols will not be converted back from a df to a  matrix object,so doing this explicitly
A2=pd.DataFrame(A, columns=explanatory_df.columns[independent_col_indexes],index=explanatory_df.index)
print explanatory_df.columns
print A2.columns
test_scores = sm.Logit(response_series,A2,missing='drop').fit()

Index([u'weight', u'height', u'total_at_bats', u'total_hits', u'total_runs', u'total_home_runs', u'total_stolen_bases', u'total_RBI', u'total_caught_stealing', u'total_hitter_strikeouts', u'total_intentional_walks', u'total_games_pitched', u'total_saves', u'total_outs_pitched', u'total_pitching_hits', u'total_pitching_earned_runs', u'total_pitcher_strikeouts', u'average_ERA', u'total_wild_pitches', u'total_hit_by_pitch', u'total_games_finished', u'total_runs_allowed', u'total_games_fielded', u'total_time_in_field_with_outs', u'total_putouts', u'total_errors', u'total_double_plays', u'teamID_ATL', u'teamID_BAL', u'teamID_BOS', u'teamID_CAL', u'teamID_CHA', u'teamID_CHN', u'teamID_CIN', u'teamID_CLE', u'teamID_DET', u'teamID_HOU', u'teamID_KCA', u'teamID_LAN', u'teamID_MIN', u'teamID_ML1', u'teamID_ML4', u'teamID_MON', u'teamID_NYA', u'teamID_NYN', u'teamID_OAK', u'teamID_Other', u'teamID_PHI', u'teamID_PIT', u'teamID_SDN', u'teamID_SFN', u'teamID_SLN', u'teamID_TEX', u'teamID_WS2', u'ba

LinAlgError: Singular matrix

In [74]:
#try removing team_df.other
results = sm.Logit(response_series,explanatory_df.drop('teamID_Other',axis=1),missing='drop').fit(maxiter=1000)


         Current function value: 0.542674
         Iterations: 1000


In [75]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               inducted   No. Observations:                  396
Model:                          Logit   Df Residuals:                      340
Method:                           MLE   Df Model:                           55
Date:                Wed, 22 Apr 2015   Pseudo R-squ.:                 -0.6307
Time:                        06:57:55   Log-Likelihood:                -214.90
converged:                      False   LL-Null:                       -131.78
                                        LLR p-value:                     1.000
=================================================================================================
                                    coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------
weight                           -0.0968      0.181     -0.536      0.592        -0.451     0.257
height                           -0.0054      0.177     -0.030      0.976        -0.353     0.342
total_at_bats                    -3.0253      1.823     -1.660      0.097        -6.597     0.547
total_hits                        3.2967      2.272      1.451      0.147        -1.156     7.749
total_runs                        1.5240      1.219      1.250      0.211        -0.865     3.913
total_home_runs                   2.5728      1.000      2.574      0.010         0.613     4.532
total_stolen_bases                0.7841      0.422      1.858      0.063        -0.043     1.611
total_RBI                        -3.4656      1.536     -2.257      0.024        -6.476    -0.456
total_caught_stealing            -0.8818      0.543     -1.624      0.104        -1.946     0.183
total_hitter_strikeouts          -0.8342      0.445     -1.875      0.061        -1.706     0.038
total_intentional_walks           1.1954      0.395      3.027      0.002         0.421     1.969
total_games_pitched              -1.2090      0.575     -2.103      0.035        -2.336    -0.082
total_saves                      -0.2122      0.575     -0.369      0.712        -1.339     0.915
total_outs_pitched                6.2868      2.436      2.581      0.010         1.512    11.062
total_pitching_hits              -3.1191      2.264     -1.378      0.168        -7.557     1.319
total_pitching_earned_runs       -6.8819      3.777     -1.822      0.068       -14.284     0.521
total_pitcher_strikeouts          0.4491      0.760      0.591      0.555        -1.041     1.939
average_ERA                       0.1220      0.138      0.885      0.376        -0.148     0.392
total_wild_pitches               -0.3072      0.277     -1.109      0.267        -0.850     0.235
total_hit_by_pitch                0.2302      0.263      0.876      0.381        -0.285     0.745
total_games_finished              1.1789      0.888      1.327      0.184        -0.562     2.919
total_runs_allowed                5.1859      4.036      1.285      0.199        -2.725    13.097
total_games_fielded               0.6581      1.029      0.640      0.522        -1.358     2.674
total_time_in_field_with_outs    -0.4975      1.022     -0.487      0.626        -2.501     1.506
total_putouts                    -0.2893      0.329     -0.880      0.379        -0.934     0.355
total_errors                      0.1582      0.382      0.415      0.678        -0.590     0.906
total_double_plays                0.2505      0.267      0.939      0.348        -0.273     0.774
teamID_ATL                       -0.2385      0.172     -1.386      0.166        -0.576     0.099
teamID_BAL                       -0.0812      0.229     -0.355      0.723        -0.529     0.367
teamID_BOS                       -0.1939      0.207     -0.939      0.348        -0.599     0.211
teamID_

In [ ]:
#Since the model still fails to converge, maybe try to model stepwise. eg. add variables one by one until it does converge, let's try all the numeric features first
exp_vars = []
